#### Imports 

In [ ]:
from time import gmtime, strftime
from utils import InferenceSpec
import sagemaker
import logging
import boto3
import json

#### Setup

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
region = sagemaker.Session().boto_region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client('sagemaker')
logger.info(f'Region = {region}')

#### List all model package groups

In [ ]:
sagemaker_client.list_model_package_groups()['ModelPackageGroupSummaryList']

### Create a Model Package Group

In [ ]:
model_package_group_name = 'BERT-Email-Classifier'
config = {'ModelPackageGroupName': model_package_group_name, 
          'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection'
         }

In [ ]:
sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']

In [ ]:
if not sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']:
    response = sagemaker_client.create_model_package_group(**config)
    logger.info(f'Model Package Group: {model_package_group_name} created!')
    logger.info(f'Response: {response}')
else:
    logger.warning('Model Package Group already exists!')

#### List Model Packages 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_model_packages

In [ ]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

#### Describe a Model Package Group

In [ ]:
sagemaker_client.describe_model_package_group(ModelPackageGroupName=model_package_group_name)

### How do I register a trained BERT model to the model registry 

In [ ]:
from sagemaker import image_uris
ecr_image = image_uris.retrieve(framework='huggingface',region='us-east-1',version='4.6.1',image_scope='inference',
                    base_framework_version='tensorflow2.4.1', py_version='py37', container_version='ubuntu18.04', 
                               instance_type='ml.m5.4xlarge')
ecr_image

1. Define the inference spec 

In [ ]:
# ecr_image = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-tensorflow-training:2.4.1-transformers4.6.1-gpu-py37-cu110-ubuntu18.04'
inference_spec = InferenceSpec().get_dict(ecr_image=ecr_image, 
                                                    supports_gpu=True,  
                                                    supported_content_types=['text/csv'], 
                                                    supported_mime_types=['text/csv'])

In [ ]:
inference_spec["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'
inference_spec

**Note:** The ModelDataUrl can also be obtained from a previously run training job.

2. Define/load model metrics 

In [ ]:
with open('./metrics.json', 'rb') as f:
    model_metrics = json.load(f)

In [ ]:
model_metrics

In [ ]:
current_timestamp = strftime('%m-%d-%H-%M', gmtime())

Let's push this metrics file to S3

In [ ]:
!aws s3 cp metrics.json s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json

In [ ]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json",
        }
    }
}

3. Create model package config (dictionary)

In [ ]:
model_package_input_dict = {"ModelPackageGroupName": model_package_group_name,
                            "ModelPackageDescription": "BERT classifier", 
                            "ModelApprovalStatus": "PendingManualApproval", 
                            "ModelMetrics": model_metrics}

model_package_input_dict.update(inference_spec)
model_package_input_dict

4. Create the Model Package 

In [ ]:
response = sagemaker_client.create_model_package(**model_package_input_dict)
response

In [ ]:
arn = response["ModelPackageArn"]
model_package_info = sagemaker_client.describe_model_package(ModelPackageName=arn)
model_package_info

In [ ]:
status = model_package_info["ModelPackageStatus"]

while status not in ["Completed", "Failed"]:
    time.sleep(5)
    info = sagemaker_client.describe_model_package(ModelPackageName=arn)
    status = info["ModelPackageStatus"]
    print(f"model package status: {status}")
print(f"model package status: {status}")

#### Describe a Model Package 

In [ ]:
arn = model_package_info['ModelPackageArn']
sagemaker_client.describe_model_package(ModelPackageName=arn)

### Deletion

#### Delete all Model Packages in a Model Package Group

List all standalone model packages (ones that are not affiliated to any Model Package Group).

In [ ]:
response = sagemaker_client.list_model_packages()
response

Delete all standalone model packages 

In [ ]:
for model_package in response['ModelPackageSummaryList']:
    print(model_package)
    #package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    print(response)

List model packages that are affiliated to our Model Package Group

In [ ]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

In [ ]:
for model_package in response['ModelPackageSummaryList']:
    print(model_package)
    #package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    print(response)

#### Delete a Model Package Group if needed

**Note:** We need to delete all Model Packages inside the Model Package Group first to delete the package group.

In [ ]:
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

#### How do I recreate the BERT model from the model registry and use it to create a real-time inference endpoint

#### How do I create an endpoint using model artifacts in S3 

#### How do I create an endpoint via Model Registry